In [6]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np 
import os

In [7]:
def carrega_face(filename, required_size=(160, 160)):
    
    #carrega as imagens
    image = Image.open(filename)
    image =  image.convert("RGB")
    
    return asarray(image)
    

In [8]:
def carrega_faces(directory_src):
    
    faces = list()

    #iteracao dos arquivos
    for filename in listdir(directory_src):
        
        path = directory_src + filename
        
        try:
            faces.append(carrega_face(path))
        except:
            print("Erro no processo de carregamento da imagem {}".format(path))
    
    return faces
        

In [9]:
def carrega_fotos(directory_src):
    X, y = list(), list()
    
    #iterar diretórios por classes
    for subdir in listdir(directory_src):
      
        path = directory_src + subdir + "\\"
        
        if not isdir(path):
            continue
            
        faces = carrega_faces(path)
        labels = [subdir for _ in range(len(faces))]
        
        #sumarização
        print("Foram carregadas %d faces da classe : %s" % (len(faces), subdir))
        
        X.extend(faces)
        y.extend(labels)
    
    return asarray(X), asarray(y)
        

In [10]:
#Diretório de localização das imagens recortadas

#trainX, trainy = carrega_fotos(directory_src="C:\\new\\dataset\\faces\\train\\") 
trainX, trainy = carrega_fotos(directory_src ="")

#print(trainX)
#print(trainy)

Carregadas 23656 faces da classe : desconhecidos


In [11]:
trainX.shape

(23656, 160, 160, 3)

In [12]:
trainy.shape

(23656,)

In [13]:
np.unique(trainy)

array(['desconhecidos'], dtype='<U13')

In [14]:
#Carregar rede neural convolucional FaceNet | Processo de normalização
from tensorflow.keras.models import load_model
model = load_model("facenet.h5")#especificar o diretório da rede

In [15]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

__________________________________________________________________________________________________
Block35_3_Branch_2_Conv2d_0b_3x (None, 17, 17, 32)   96          Block35_3_Branch_2_Conv2d_0b_3x3[
__________________________________________________________________________________________________
Block35_3_Branch_1_Conv2d_0a_1x (None, 17, 17, 32)   0           Block35_3_Branch_1_Conv2d_0a_1x1_
__________________________________________________________________________________________________
Block35_3_Branch_2_Conv2d_0b_3x (None, 17, 17, 32)   0           Block35_3_Branch_2_Conv2d_0b_3x3_
__________________________________________________________________________________________________
Block35_3_Branch_0_Conv2d_1x1 ( (None, 17, 17, 32)   8192        Block35_2_Activation[0][0]       
__________________________________________________________________________________________________
Block35_3_Branch_1_Conv2d_0b_3x (None, 17, 17, 32)   9216        Block35_3_Branch_1_Conv2d_0a_1x1_
__________

Block8_6_Branch_1_Conv2d_0c_3x1 (None, 3, 3, 192)    110592      Block8_6_Branch_1_Conv2d_0b_1x3_A
__________________________________________________________________________________________________
Block8_6_Branch_0_Conv2d_1x1_Ba (None, 3, 3, 192)    576         Block8_6_Branch_0_Conv2d_1x1[0][0
__________________________________________________________________________________________________
Block8_6_Branch_1_Conv2d_0c_3x1 (None, 3, 3, 192)    576         Block8_6_Branch_1_Conv2d_0c_3x1[0
__________________________________________________________________________________________________
Block8_6_Branch_0_Conv2d_1x1_Ac (None, 3, 3, 192)    0           Block8_6_Branch_0_Conv2d_1x1_Batc
__________________________________________________________________________________________________
Block8_6_Branch_1_Conv2d_0c_3x1 (None, 3, 3, 192)    0           Block8_6_Branch_1_Conv2d_0c_3x1_B
__________________________________________________________________________________________________
Block8_6_C

In [16]:
#função geradora de embeddings | Processo de incorporação das imagens
def get_embedding(model, face_pixels):
    #padronização
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    #exemplo: transformar a face em ...
    #(160,160) -> (1,160,160)
    
    samples = expand_dims(face_pixels, axis=0) 
    yhat = model.predict(samples)
    
   # [[1,2..128], [1,2,128]]
    return yhat[0]
    

In [17]:
#lista dos embedding
newTrainX = list()

In [18]:
type(trainX)

numpy.ndarray

In [19]:
for face_pixels in trainX:
    
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)

In [20]:
newTrainX = asarray(newTrainX)

In [21]:
newTrainX.shape

(23656, 128)

In [22]:
import pandas as pd

In [23]:
data = pd.DataFrame(data = newTrainX)

In [24]:
data

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-1.184488,-0.037075,-0.310449,1.535288,-0.426154,-0.164261,-1.946720,-1.504363,0.544507,2.118238,...,0.259567,-0.148841,1.511575,-1.754456,0.349307,-1.310328,-0.277957,-0.487598,3.974886,1.516967
1,-1.729603,-2.216308,2.378981,0.020642,-1.470711,0.078194,1.397792,-0.148966,1.196654,-0.129112,...,0.522989,0.274649,-0.213524,0.412351,0.239262,-0.517173,0.077622,0.423780,1.126909,-0.979598
2,-0.115113,-1.064566,-0.331112,0.927957,-2.399568,1.120506,-1.681376,0.091642,-0.008483,1.015339,...,-1.087530,1.456092,-0.575175,0.841954,-0.035182,-1.049374,0.678325,-0.883975,0.814864,-1.241990
3,-0.177592,0.574779,-1.185092,1.978255,-0.274014,0.965762,-0.085740,-0.403561,0.488117,0.852208,...,1.733690,-1.022199,1.229965,-1.997093,0.548645,-0.100363,0.511239,0.074036,2.325175,-0.177879
4,0.825329,1.542661,0.146476,-0.159960,-0.283281,-1.694085,-0.490159,-1.559936,-0.818318,-0.511240,...,1.753447,-0.732255,-0.139215,-0.944376,1.076449,-1.135574,0.729563,-1.979146,0.163751,0.525420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23651,-0.068415,0.543134,1.673895,2.066742,0.158463,-0.857751,-0.168971,0.811612,2.548283,-0.958622,...,1.121886,-1.766712,-0.155642,-0.158090,-1.735419,-1.828769,0.110674,0.346123,-1.384761,1.436030
23652,1.956248,1.634966,0.591942,0.192853,1.183304,0.023359,0.436231,-1.742470,-0.382186,0.710666,...,-1.496928,-0.447796,0.747839,-0.503061,1.850725,-0.991245,0.410959,0.166534,-0.286393,0.681391
23653,0.074280,0.848793,-1.085095,0.925459,-0.671459,-0.898966,0.318714,-0.427897,0.857371,-0.088732,...,2.257524,0.183821,0.630258,0.644186,-0.915037,-1.832088,-0.500535,-0.303524,1.059625,-0.888787
23654,-0.486035,0.995566,-0.085366,1.655110,-1.119586,0.738832,0.096784,-0.254087,1.136138,-0.116009,...,2.432756,-0.521814,-0.001916,0.275389,0.760365,-1.199390,0.755312,-1.235536,2.205646,-0.609340


In [25]:
data['target']=trainy #rótulos das imagens/classes

In [26]:
data

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.184488,-0.037075,-0.310449,1.535288,-0.426154,-0.164261,-1.946720,-1.504363,0.544507,2.118238,...,-0.148841,1.511575,-1.754456,0.349307,-1.310328,-0.277957,-0.487598,3.974886,1.516967,desconhecidos
1,-1.729603,-2.216308,2.378981,0.020642,-1.470711,0.078194,1.397792,-0.148966,1.196654,-0.129112,...,0.274649,-0.213524,0.412351,0.239262,-0.517173,0.077622,0.423780,1.126909,-0.979598,desconhecidos
2,-0.115113,-1.064566,-0.331112,0.927957,-2.399568,1.120506,-1.681376,0.091642,-0.008483,1.015339,...,1.456092,-0.575175,0.841954,-0.035182,-1.049374,0.678325,-0.883975,0.814864,-1.241990,desconhecidos
3,-0.177592,0.574779,-1.185092,1.978255,-0.274014,0.965762,-0.085740,-0.403561,0.488117,0.852208,...,-1.022199,1.229965,-1.997093,0.548645,-0.100363,0.511239,0.074036,2.325175,-0.177879,desconhecidos
4,0.825329,1.542661,0.146476,-0.159960,-0.283281,-1.694085,-0.490159,-1.559936,-0.818318,-0.511240,...,-0.732255,-0.139215,-0.944376,1.076449,-1.135574,0.729563,-1.979146,0.163751,0.525420,desconhecidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23651,-0.068415,0.543134,1.673895,2.066742,0.158463,-0.857751,-0.168971,0.811612,2.548283,-0.958622,...,-1.766712,-0.155642,-0.158090,-1.735419,-1.828769,0.110674,0.346123,-1.384761,1.436030,desconhecidos
23652,1.956248,1.634966,0.591942,0.192853,1.183304,0.023359,0.436231,-1.742470,-0.382186,0.710666,...,-0.447796,0.747839,-0.503061,1.850725,-0.991245,0.410959,0.166534,-0.286393,0.681391,desconhecidos
23653,0.074280,0.848793,-1.085095,0.925459,-0.671459,-0.898966,0.318714,-0.427897,0.857371,-0.088732,...,0.183821,0.630258,0.644186,-0.915037,-1.832088,-0.500535,-0.303524,1.059625,-0.888787,desconhecidos
23654,-0.486035,0.995566,-0.085366,1.655110,-1.119586,0.738832,0.096784,-0.254087,1.136138,-0.116009,...,-0.521814,-0.001916,0.275389,0.760365,-1.199390,0.755312,-1.235536,2.205646,-0.609340,desconhecidos


In [27]:
# Salvar o Dataset/Embedding
data.to_csv('faces_treino_unknow_23656.csv', index=False)